In [1]:
import sys
print(sys.executable)

/anaconda3/envs/magpie/bin/python


In [ ]:
import os
import json
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation, Dropout
from keras import optimizers

%matplotlib inline
pd.options.display.max_columns = 999

In [ ]:
def add_time_features(df):
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='ignore')
    df['year'] = df['date'].apply(lambda x: x.year)
    df['month'] = df['date'].apply(lambda x: x.month)
    df['day'] = df['date'].apply(lambda x: x.day)
    df['weekday'] = df['date'].apply(lambda x: x.weekday())
    
    return df

In [ ]:
def load_df(csv_path='/Users/amitadate/WORK/MSAI/349_machine_learning/project_notebook/train_v2.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [ ]:
train = load_df("train_v2.csv")
test = load_df("test_v2.csv")

In [ ]:
print('TRAIN SET')
print('Rows: %s' % train.shape[0])
print('Columns: %s' % train.shape[1])
print('Features: %s' % train.columns.values)
print()
print('TEST SET')
print('Rows: %s' % test.shape[0])
print('Columns: %s' % test.shape[1])
print('Features: %s' % test.columns.values)

In [ ]:
train = add_time_features(train)
test = add_time_features(test)
# Convert target feature to 'float' type.
train['totals.transactionRevenue'] = train["totals.transactionRevenue"].astype('float')
train['totals.pageviews'] = train['totals.pageviews'].astype(float)
test['totals.pageviews'] = test['totals.pageviews'].astype(float)
test['totals.transactionRevenue'] = test["totals.transactionRevenue"].astype('float')

In [ ]:
# dropiing the unwanted columns
unwanted_columns = ['customDimensions', 'day', 'hits']

In [ ]:
train = train.drop(unwanted_columns, axis=1)
test = test.drop(unwanted_columns, axis=1)

In [ ]:
constant_columns = [c for c in train.columns if train[c].nunique()<=1]
print('Columns with constant values: ', constant_columns)

In [ ]:
train = train.drop(constant_columns, axis=1)

In [ ]:
constant_columns = [c for c in test.columns if test[c].nunique()<=1]
print('Columns with constant values: ', constant_columns)

In [ ]:
test = test.drop(constant_columns, axis=1)

In [ ]:
train = train.drop('year', axis=1)